<div align="center">

# **Régression Logistique sur un Dataset Réel** 

</div>
<br><br>
<br><br>

## **Introduction**

Dans ce volet, il s'agira de confronter les versions "maison" de nos algorithmes de descente de gradient et de prédiction à un jeu de données du réel. Ce jeu de données est un dataset issu d'une banque portugaise, dont l'objectif est d'identifier les clients les plus enclins à souscrire un produit financier. En plus d'être plus volumineux qu'un dataset académique, un jeu de données issu du terrain peut challenger la descente de gradient de par sa plus grande variabilité et son bruit, surtout en présence d'un déséquilibre entre les classes.

Une première partie consistera à reproduire les mêmes traitements et choix que la démonstration, afin d'aboutir à un dataset ajusté et exploitatble par nos algorithmes.

Dans la deuxième partie, la régression logistique sera appliquée avec les méthodes d'optimisation implémentées dans le Bloc-1, auxquelles ont été rajoutées deux implémentations de descente de gradient pénalisées, La ridge et la Lasso.

## **I- Reproduction des traitements et choix de variables dans le dataset**

Cette partie se contente d'acquérir le dataset et de lui appliquer les mêmes traitements que dans la démonstration.

In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import statsmodels.api as sm
from IPython.display import display
from sklearn.metrics import roc_curve, auc


from Optimisations import * # fichier Optimisations.py: Nos descentes de gradients élaborés en bloc-1 + DG pénalisée Ridge + DG pénalisée Lasso
from Outils import * # fichier Outils.py: les fonctions de prédiction, des outils de plot..

#### **DATA**

In [6]:
data = pd.read_csv('bank.csv', header=0)
data = data.dropna()
print(data.shape)
print(list(data.columns))

In [7]:
data.dtypes

In [8]:
data.head()

#### **Grouping Columns**

In [10]:
data['education'].unique()

In [11]:
data['education']=np.where(data['education'] =='basic.9y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.6y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.4y', 'Basic', data['education'])

In [12]:
data['education'].unique()

#### **Exploration**

In [14]:
data['y'].value_counts()

In [15]:
sns.countplot(x='y', data=data, palette='hls', hue='y')
plt.show()
plt.savefig('count plot')

In [16]:
count_no_sub = len(data[data['y']==0])
count_sub = len(data[data['y']==1])
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("percentage of no subscription is", pct_of_no_sub*100)
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("percentage of subscription", pct_of_sub*100)

In [17]:
data.groupby('y').mean(numeric_only=True)

In [18]:
data.groupby('job').mean(numeric_only=True)

In [19]:
data.groupby('marital').mean(numeric_only=True)

In [20]:
data.groupby('education').mean(numeric_only=True)

In [21]:
%matplotlib inline
pd.crosstab(data.job,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Job Title')
plt.xlabel('Job')
plt.ylabel('Frequency of Purchase')
plt.savefig('purchase_fre_job')

In [22]:
table=pd.crosstab(data.marital,data.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Marital Status vs Purchase')
plt.xlabel('Marital Status')
plt.ylabel('Proportion of Customers')
plt.savefig('mariral_vs_pur_stack')

In [23]:
table=pd.crosstab(data.education,data.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Education vs Purchase')
plt.xlabel('Education')
plt.ylabel('Proportion of Customers')
plt.savefig('edu_vs_pur_stack')

In [24]:
pd.crosstab(data.day_of_week,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_dayofweek_bar')

In [25]:
pd.crosstab(data.month,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Month')
plt.xlabel('Month')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_fre_month_bar')

In [26]:
data.age.hist()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.savefig('hist_age')

In [27]:
pd.crosstab(data.poutcome,data.y).plot(kind='bar')
plt.title('Purchase Frequency for Poutcome')
plt.xlabel('Poutcome')
plt.ylabel('Frequency of Purchase')
plt.savefig('pur_fre_pout_bar')

#### **Transformation Dummy variables**

In [29]:
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(data[var], prefix=var)
    cat_list = cat_list.astype(int)
    data1=data.join(cat_list)
    data=data1
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
data_vars=data.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [30]:
# Trouver toutes les colonnes de type booléen et les convertir en int
data[data.select_dtypes(['bool']).columns] = data.select_dtypes(['bool']).astype(int)

data.dtypes[:50]

In [31]:
data_final=data[to_keep]
data_final.columns.values

#### **Oversampling using SMOTE**

In [33]:
X = data_final.loc[:, data_final.columns != 'y']
y = data_final.loc[:, data_final.columns == 'y']
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

#### **Elimination des features les moins pertinentes**

In [35]:
cols=['euribor3m', 'job_blue-collar', 'job_housemaid', 'marital_unknown', 'education_illiterate', 
      'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar', 
      'month_may', 'month_nov', 'month_oct', "poutcome_failure", "poutcome_success"] 
X=os_data_X[cols]
y=os_data_y['y']
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

#### **Modèle de l'exemple**

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(
    C=1.0, 
    class_weight=None, 
    dual=False, 
    fit_intercept=True, 
    intercept_scaling=1, 
    max_iter=100, 
    multi_class='ovr', 
    n_jobs=1, 
    penalty='l2', 
    random_state=None, 
    solver='liblinear', 
    tol=0.0001, 
    verbose=0, 
    warm_start=False
)

logreg.fit(X_train, y_train)

In [38]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

## **II- Régressions et résultats**

Appliquons nos algorithmes de descente à ce dataset:

In [40]:
# paramètres 
n_features = X_train.shape[1]
w_random = random_initialize_parameters(n_features)
learning_rate = 0.5
num_iterations = 1000
batch_size = 256
momentum = 0.9
epsilon = 1e-8
lambda_reg = 0.1  # Pour Ridge et Lasso


# N'oublions pas que nos algorithmes fonctionnent avec des matrics et que nous avons des dataframes:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

X_test = X_test.to_numpy()
y_test = y_test.to_numpy()




In [41]:
# descente de gradient simple:
w_gd, costs_gd = gradient_descent(X_train, y_train, w_random, learning_rate, num_iterations)

In [42]:
# descente de gradient stochastique :
w_sgd, costs_sgd = stochastic_gradient_descent(X_train, y_train, w_random, learning_rate, num_epochs=8, batch_size=256)


In [43]:
# descente de gradient avec momentum
w_momentum, costs_momentum = gradient_descent_momentum(X_train, y_train, w_random, learning_rate, num_iterations=1000)

In [44]:
# descente de gradient Nesterov
w_nesterov, costs_nesterov = gradient_descent_nesterov(X_train, y_train, w_random, learning_rate, num_iterations)

In [45]:
# descente de gradient adagrad
w_adagrad, costs_adagrad = gradient_descent_adagrad(X_train, y_train, w_random, learning_rate, num_iterations)

In [46]:
# descente de gradient RMSprop
w_rmsprop, costs_rmsprop = gradient_descent_rmsprop(X_train, y_train, w_random, learning_rate, num_iterations)

In [47]:
# descente de gradient Adam
w_adam, costs_adam = gradient_descent_adam(X_train, y_train, w_random, learning_rate, num_iterations)

In [48]:
# descente de gradient avec régularisation ridge
w_ridge, costs_ridge = gradient_descent_ridge(X_train, y_train, w_random, learning_rate, num_iterations, lambda_reg)

In [49]:
# descente de gradient avec régularisation Lasso
w_lasso, costs_lasso = gradient_descent_lasso(X_train, y_train, w_random, learning_rate, num_iterations, lambda_reg)

In [50]:
# regroupons en dictionnaire nos résultats
results_costs = {
    'Gradient Descent': costs_gd,
    'SGD': costs_sgd,
    'Momentum': costs_momentum,
    'Nesterov': costs_nesterov,
    'AdaGrad': costs_adagrad,
    'RMSProp': costs_rmsprop,
    'Adam': costs_adam,
    'Ridge': costs_ridge,
    'Lasso': costs_lasso
}

results_weights = {
    'Gradient Descent': w_gd,
    'SGD': w_sgd,
    'Momentum': w_momentum,
    'Nesterov': w_nesterov,
    'AdaGrad': w_adagrad,
    'RMSProp': w_rmsprop,
    'Adam': w_adam,
    'Ridge': w_ridge,
    'Lasso': w_lasso
}


#### **1- Courbes de convergence de la perte:**

Comme le montre le graphe suivant, nos différentes méthodes de descente de gradient, ont toutes convergé avec une performance de rapidité notoire pour la descente de gradient Adam.

In [52]:
exclure = ["RMSProp"]
plot_losses(results_costs, index_min=0, index_max=500, dontplot=exclure)

Nous pouvons néanmoins constater que La descente de gradient RMSPop a pâti d'une instabilité, ne l'ayant pas empêché de converger, mais l'ayant contraint à osciller entre des valeurs de 0.45 et 0.5. Soit une amplitude de 0.05 en unité de perte.

In [54]:
inclure = ["RMSProp"]
plot_losses(results_costs, index_min=0, index_max=500, plot=inclure)
plot_losses(results_costs, index_min=900, index_max=1000, plot=inclure)

#### **2- Prédictions avec les poids optimaux obtenus par chaque méthode:** 

Essayons d'utiliser nos poids maintenant, afin de prédire les labels sur le set de test

In [57]:
# Dictionnaire pour stocker les prédictions des labels pour chaque algorithme
results_labels = {}

# Seuil de 0.5
threshold = 0.5

# Appliquer la prédiction pour chaque algorithme dans results_weights
for algo, weights in results_weights.items():
    # Calculer les probabilités avec sigmoid
    probabilities = predict(X_test, weights)
    
    # Appliquer le seuil de 0.5 pour obtenir les prédictions en 0 ou 1
    labels = (probabilities >= threshold).astype(int)
    
    # Stocker les labels dans le dictionnaire results_labels
    results_labels[algo] = labels

# Affichage des résultats de l'accuracy (nombre de bonnes réponses sur tout le dataset de test)
for algo, labels in results_labels.items():
    accuracy = np.mean(y_test == labels)
    print(f"Accuracy de l'algorithme {algo}: {accuracy:.4f}")  

Nous avons globalement de très bons résultats en accuracy avec tous nos algorithmes. La différence sur un même résultat d'accuracy a résidé dans la vitesse de convergence, qui a eu lieu à partir de la 30ième itération pour Adam, et a coûté significativement plus d'itérations pour les autres algorithmes.

#### **2- Calcul des différentes métriques pour chaque algorithme** 

Pour éviter la saturation en code dans le notebook, les fonctions "maison" de calcul des différentes métriques ainsi que les fonctions d'affichage sont dans le fichier **Outils.py**. Voici le tableau récapitulatif des différentes métriques pour chaque type de descente de gradient:

In [60]:
# Calcul des métriques (accuracy, precision, recall et F1-score) pour toutes les descentes
all_algos_metrics = calculate_all_metrics_for_all(y_test, results_labels)
# Affichage
display_metrics_dataframe(all_algos_metrics)

Nos algorithmes montrent de bons résultats sur les différentes métriques, en particulier:

- Gradient Descent, Ridge, et Lasso se distinguent avec la meilleure combinaison de précision et de rappel, offrant une F1 Score de quasiment 0.829560, ce qui les rend bien adaptés si on cherche un bon compromis entre ces deux métriques.

  
- RMSProp est l'algorithme le plus déséquilibré, avec une haute précision mais un rappel très faible.

  
- SGD, Momentum, Nesterov, AdaGrad, et Adam sont compétitifs, avec des performances relativement similaires, mais Adam se démarque légèrement pour sa robustesse globale.

Pour avoir une idée plus claire, voici leurs matrices de confusion:

In [62]:
plot_confusion_matrices(all_algos_metrics)

Ces résultats sont à interpréter à la lumière de l'objectif. Ici, il est d'identifier de potentiels clients d'un produit financier :

- Faux positifs: on prédit qu'un client est intéressé (ou potentiel) alors qu'il ne l'est pas réellement.

  
Impact : l'entreprise dépense des ressources (temps, argent, efforts marketing) sur des clients qui ne sont finalement pas intéressés. Cela peut représenter un coût financier mais n'a pas d'impact direct sur les opportunités de revenu.

- Faux négatifs: on prédit qu'un client n'est pas intéressé, alors qu'il l'est.
  
Impact : l'entreprise manque une opportunité de convertir un client potentiel, ce qui peut entraîner une perte de revenu. En plus, rater un client revient à renforcer la concurrence qui la capte (effet double).


L'erreur à minimiser dans ce contexte est l'erreur de type 2, c'est à dire les faux négatifs. Dans un contexte commercial, cette erreur est souvent plus coûteuse car l'acquisition de nouveaux clients est généralement une priorité. 

la descente de gradient stochastique montre de très bons résultats pour le recall, tout en gardant un bon compromis f1-score. Elle serait apriori la plus adaptée pour cette problématique.

Visualisons sa courbe ROC-AUC :


In [64]:
# calculer et afficher la courbe ROC-AUC
probabilities_sgd = predict(X_test, w_sgd)
auc_sgd = plot_roc_curve(y_test, probabilities_sgd)
print(f"AUC pour la descente de gradient stochastique: {auc_sgd:.2f}")

L'aire sous la courbe est de 0,89, ce qui signifie que la probabilité qu'un échantillon positif ait un score prédictif supérieur à celui d'un échantillon négatif, est de 89 % :

$\text{AUC} = P(\text{Score}_{\text{positif}} > \text{Score}_{\text{négatif}}) = 0{,}89$

où :
* $P$ représente ici la probabilité.
* $\text{Score}_{\text{positif}}$ est le score ou la probabilité attribué par le modèle à un échantillon positif.
* $\text{Score}_{\text{négatif}}$ est le score ou la probabilité attribué à un échantillon négatif.

Ce qui représente une performance très acceptable du modèle de descente de gradient stochastique dans cet exercice.

## **Conclusion**

Cette partie expérimentale sur un dataset réel de marketing bancaire a permis de mettre en pratique et comparer différentes méthodes d'optimisation qu'on a implémenté pour la régression logistique :

- Les algorithmes implémentés (descente de gradient simple, stochastique, avec momentum, Nesterov, AdaGrad, RMSprop, Adam, et régularisations Ridge/Lasso) ont tous convergé efficacement, avec des performances légèrement différentes.
  
- Les méthodes adaptatives (Adam, RMSprop) et celles basées sur le momentum ont montré les meilleures vitesses de convergence.

  
- Toutes les méthodes ont atteint une précision élevée (> 80%) sur ce jeu de données, la SGD se démarquant légèrement avec le meilleur rappel, et donc la meilleure capacité à minimiser le nombre de faux négatifs.

Ce taux élevé de bonnes prédictions est à relativiser certainement par le fait que ça reste un dataset de benchmarking, et qu'il exite probablement des spécificités aux variables choisis qui demanderaient plus de recherches.

En tous cas, Ces résultats démontrent l'efficacité des méthodes implémentées et leur capacité à solutionner ce problème complexe qui est l'optimisation d'une fonction de perte. 

Dans ce travail, le focus a été mis sur une fonction de perte "Convexe", sur laquelle les algorithmes d'optimisation, en particulier de descente de gradient restent globalement efficaces et prévisisbles. 
Cependant, la réalité des problèmes complexes d'apprentissage automatique dépasse souvent ce cadre simple et bien défini. De nombreux modèles performants, tels que les réseaux de neurones profonds, reposent sur des fonctions de perte non convexes, où les défis d'optimisation deviennent nettement plus complexes.